## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [Medical LLM - Medium](https://aws.amazon.com/marketplace/pp/prodview-z4jqmczvwgtby)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.


- **Model**: `jsl-medical-summarization-qa-70b`
- **Model Description**: Medical model for summarization, question answering (open-book and closed-book), and general chat.

In [5]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [6]:
import os
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [7]:
model_name = "jsl-medical-summarization-qa-70b"

real_time_inference_instance_type = "ml.g6.48xlarge"
batch_transform_inference_instance_type = "ml.g5.48xlarge"

## 2. create a deployable model from the model package.

In [8]:
# Define ModelPackage
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn, 
    sagemaker_session=sagemaker_session, 
)

### Important Parameters

**Required Parameter:**
- **`input_text`**: The input text(s) provided to the model.
  - **Type**: `Union[str, dict, List[str], List[dict]]`
  - **Constraints**: Provide a string for a single text or a list of strings for multiple inputs. Additionally, you can provide a dictionary or a list of dictionaries with keys `context` and `question` with `open_book_qa`, and the input will be formatted accordingly before sending to the model. If no template is selected, the `context` and `question` strings will be concatenated with a newline.

**Optional Parameters (Inside `params` Key):**
- **`max_tokens`**: The maximum number of tokens the model should generate as output.
  - **Type**: `int`
  - **Default**: `1024`
  - **Constraints**: Must be a positive integer greater than 0.

- **`temperature`**: The temperature parameter controlling the randomness of token generation.
  - **Type**: `float`
  - **Default**: `0.7`
  - **Constraints**: Must be a float greater than or equal to 0.

- **`repetition_penalty`**: Penalizes new tokens based on whether they appear in the prompt and generated text so far. Values > 1 discourage repetition, while values < 1 encourage it.
  - **Type**: `float`
  - **Default**: `1.2`
  - **Constraints**: Must be a float in the range (0.0, 2.0].

- **`top_p`**: Controls nucleus sampling by selecting tokens with a cumulative probability of `top_p`.
  - **Type**: `float`
  - **Default**: `0.9`
  - **Constraints**: Must be a float in (0, 1].

- **`top_k`**: Limits the token selection to the `top_k` most likely tokens.
  - **Type**: `int`
  - **Default**: `50`
  - **Constraints**: Must be -1 (disable) or a positive integer.

- **`template`**: Specifies the predefined template to apply to the input text.
  - **Type**: `str`
  - **Default**: `None`

**Templates Available:**
```json
{
    "summarization": "Summarize the following document:\n## Document Start ##\n{context}\n## Document End ##",
    "open_book_qa": "Answer the following question based on the given context:\n## Context Start ##\n{context}\n## Context End ##\n## Question Start ##\n{question}\n## Question End ##",
    "closed_book_qa": "Answer the following question:\n## Question Start ##\n{question}\n## Question End ##"
}
```
If no template is provided, the `input_text` is used directly without formatting.

> **Parameter Priority**: User-provided parameters take precedence, followed by environment variables, and then default values.

---

### Model Configuration

| Parameter                  | Value     | Description                                                                                                                                                                                                                                                                                                                               |
|----------------------------|-----------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **`dtype`**                | `float16` | The data type for the model weights and activations.                                                                                                                                                                                                                                                                                      |
| **`max_model_len`**        | `8192`  | This indicates that your input and the model response combined should come under this limit (`input + output <= max_model_len`). |
| **`tensor_parallel_size`** | `8`     | The number of GPUs to use for distributed execution with tensor parallelism.                                                                                                                                                                                                                                                              |

Other than the parameters mentioned above, we are utilizing the default parameters specified for the `LLM` class in the [VLLM documentation](https://docs.vllm.ai/en/latest/dev/offline_inference/llm.html).

---

### Input Format

To use the model, provide input in one of the following formats:

#### Format 1: Single Input
Provide a single input as a JSON object.

```json
{
    "input_text": "input text",
    "params": {
        "max_tokens": 1024,
        "temperature": 0.7
    }
}
```

Alternatively, provide a dictionary with `context` and `question` keys.

```json
{
    "input_text": {"context": "Context", "question": "Question"},
    "params": {
        "max_tokens": 1024,
        "template": "open_book_qa"
    }
}
```

#### Format 2: Multiple Inputs
Provide multiple inputs as a JSON array.

```json
{
    "input_text": [
        "input text 1",
        "input text 2"
    ],
    "params": {
        "max_tokens": 1024,
        "temperature": 0.7
    }
}
```

Alternatively, provide an array of dictionaries with `context` and `question` keys.

```json
{
    "input_text": [
        {"context": "Context 1", "question": "Question 1"},
        {"context": "Context 2", "question": "Question 2"}
    ],
    "params": {
        "max_tokens": 1024,
        "template": "open_book_qa"
    }
}
```

**Note**: If you provide dictionaries, you can specify the template as `open_book_qa`. If no template is selected, `context` and `question` are concatenated with a newline.

#### Format 3: JSON Lines (JSONL)
Provide input in JSON Lines format, where each line is a JSON object.

```
{"input_text": "input text 1", "params": {"max_tokens": 1024}}
{"input_text": "input text 2", "params": {"max_tokens": 512}}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [ ]:
# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
    model_data_download_timeout=3600
)

In [10]:
import time

def invoke_realtime_endpoint(record):

    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType="application/json",
        Accept="application/json",
        Body=json.dumps(record),
    )

    return json.load(response["Body"])

### Question Answering

#### Initial setup

In [11]:
open_book_qa = [
    {
        "context": "Within the last several days, he has seen you in clinic and once again discussed whether or not to undergo radiation for his left temporal lesion. The patient has clearly been extremely ambivalent about this therapy for reasons that are not immediately apparent. It is clear that his MRI is progressing and that it seems unlikely at this time that anything other than radiation would be particularly effective. Despite repeatedly emphasizing this; however, the patient still is worried about potential long-term side effects from treatment that frankly seem unwarranted at this particular time",
        "question": "What is the patient's concern regarding undergoing radiation therapy for his left temporal lesion?"
    },
    {
        "context": "The patient is a 52-year-old male who is here for independent medical evaluation. The patient states that he works for ABC ABC as a temporary worker. He worked for ABCD too. The patient's main job was loading and unloading furniture and appliances for the home. The patient was approximately there for about two and a half weeks. Date of injury occurred back in October. The patient stating that he had history of previous back problems ongoing; however, he states that on this particular day back in October, he was unloading an 18-wheeler at ABC and he was bending down picking up boxes to unload and load. Unfortunately at this particular event, the patient had sharp pain in his lower back. Soon afterwards, he had radiating symptoms down his right buttock all the way down to the lateral part of his leg crossing his knee. This became progressively worse", 
        "question": "What symptoms did the patient experience after the injury?"
    },
    {
        "context": "Patient with a past medical history of hypertension for 15 years. (Medical Transcription Sample Report) HISTORY OF PRESENT ILLNESS: The patient is a 74-year-old white woman who has a past medical history of hypertension for 15 years, history of CVA with no residual hemiparesis and uterine cancer with pulmonary metastases, who presented for evaluation of recent worsening of the hypertension. According to the patient, she had stable blood pressure for the past 12-15 years on 10 mg of lisinopril.", 
        "question": "What is the primary issue reported by the patient?"
    }
]


closed_book_qa = """
A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus.

Which of the following is the best treatment for this patient?
A: Ampicillin
B: Ceftriaxone
C: Ciprofloxacin
D: Doxycycline
E: Nitrofurantoin
"""


#### Example: 1

In [12]:
input_data = {
    "input_text": closed_book_qa,
    "params": {"template": "closed_book_qa"}
}

data  = invoke_realtime_endpoint(input_data)
pd.DataFrame(data)

,response
0,The best answer is E) Nitrofurantoin.


#### Example: 2

In [14]:
input_data = {
    "input_text": open_book_qa,
    "params": {"template": "open_book_qa"}
}
# The timeout for InvokeEndpoint is 60 seconds, so please carefully select the number of input texts you are sending in one request.
data = invoke_realtime_endpoint(input_data)
pd.DataFrame(data)

,response
0,The patient is concerned about potential long-term side effects of radiation therapy.
1,"After the injury, the patient experienced sharp pain in his lower back which then led to radiating symptoms down his right buttock and extending to the lateral part of his leg, crossing over his knee. These symptoms worsened over time."
2,The primary issue reported by the patient is the recent worsening of her hypertension.


### Summarization

#### Initial setup

In [16]:
# Articles related to COVID-19 from the CORD-19 dataset
covid19_articles = [
    """INTRODUCTION: Infection with the new coronavirus responsible for Severe Acute Respiratory Syndrome (SARS-CoV-2) continues to spread worldwide. In Brazil, there are already more than 230 thousand dead, many of these older adults. OBJECTIVE: To present the clinical characteristics of older Brazilian adults infected by COVID-19, in the epidemiological weeks (EW) 34-52, and to verify factors responsible for the increased risk of death. METHODS: Retrospective and observational study conducted with secondary publicly available data, provided by the Brazilian Ministry of Health. 1,544 confirmed cases of registered COVID-19 infection were included between August 16 and December 26, 2020, aged 60 or older. OUTCOMES: Demographic data, comorbidity, symptoms for disease, clinical information: days of hospitalization, chest X-ray, type of RT-PCR. RESULTS: 48% of patients admitted to the ICU with evidence for SARS-CoV-2 died. Symptoms and comorbidities related to increased chance of death (OR) were immunodeficiency (188%), kidney disease (166%), neurological disease (103%), dyspnea (86%), pneumopathy (55%), O2 saturation <95% (53%), respiratory discomfort (49%), age (36%), sore throat (31%), and sex (0.5%). There was a 5% increase in the chance of death for each year of life. CONCLUSION: Heart disease and Diabetes mellitus were the most frequent comorbidities, but did not indicate an increased risk of death from SARS-CoV-2 infection. Age, sex, sore throat, dyspnea, respiratory discomfort, O2 saturation <95%, neurological disease, pneumopathy, immunodeficiency, and kidney disease were significantly associated with risk of death from COVID-19.""",
    """OBJECTIVE: To compare maternal and perinatal outcomes between randomized trials and observational studies in which conservative management was performed for more than 48 h in patients with early-onset severe preeclampsia. METHODOLOGY: We searched PubMed, LILACS, Cochrane and Google Scholar. The studies were divided in two groups: randomized and observational studies, from 1990 to 2018 that included patients with severe preeclampsia before 34 weeks of gestation with pregnancy prolongation &#8805;48 h but that did not include fetal growth restriction or HELLP syndrome at the beginning. The main variables recorded were maternal and perinatal complications. MAIN RESULTS: Forty-four studies met the inclusion criteria, and 5 of these were randomized. The average pregnancy prolongation was 9 days, with no difference between groups. Maternal complications were significantly more common in observational studies, RR = 0.71, 95% CI (0.54-0.93), p = .009. Perinatal complications were also significantly more common in observational studies (RR = 0.89, 95% CI (0.80-0.98), p = .01) at the expense of stillbirth and neonatal deaths. The percentages of cesarean sections were significantly higher in randomized studies, RR = 1.54, 95% CI (1.46-1.64). There were 2 maternal deaths, both in observational studies. CONCLUSION: Observational studies in which conservative management of early-onset preeclampsia is performed and do not include patients with fetal growth restriction or patients with HELLP syndrome and where at least 2 days of pregnancy prolongation is achieved are associated with significantly more maternal and perinatal complications.""",
    """Background:In recent years, an integrated programme of vector control, screening and treatment of gambiense human African trypanosomiasis (gHAT) infections has led to a rapid decline in cases in the Mandoul disease focus of Chad. In this study, we assess whether elimination of transmission has already been achieved in the region despite low-level case reporting, quantify the role of intensified interventions in transmission reduction, and predict the trajectory of gHAT in Mandoul for the next decade under a range of control scenarios. Method: We utilise human case data (2000-2019) to update a previous model of transmission of gHAT in Mandoul. We also test the updated model, which now has refined assumptions on diagnostic specificity of the current algorithm and an improved fitting method, via a data censoring approach.Results:We conclude that passive detection rates have increased due to improvements in diagnostic availability in fixed health facilities since 2015, by 2.1-fold for stage 1 detection, and by 1.5-fold for stage 2. We find that whilst the diagnostic algorithm for active screening is estimated to be highly specific (99.93%, 95% CI: 99.91-99.95%), the high screening level and limited remaining infection means that some recently reported cases might be false positives, especially the ones that were not parasitologically confirmed. We also find that the focus-wide tsetse vector reduction estimated through model fitting (99.1%, 95% CI: 96.1-99.6%) is comparable to the very high reduction previously measured by the decline in catches of tsetse from monitoring traps. In line with previous results, the model suggests that transmission was likely interrupted in 2015 as a result of intensified interventions. Conclusions: We recommend that additional confirmatory testing is performed in Mandoul in order that the endgame can be carefully monitored now that infection levels are so low. More specific measurement of cases would better inform when it is safe to stop active screening and vector control.""",
    """Microanatomy of the vast majority of human organs at birth is characterized by marked differences as compared to adult organs, regarding their architecture and the cell types detectable at histology. In preterm neonates, these differences are even more evident, due to the lower level of organ maturation and to ongoing cell differentiation. One of the most remarkable finding in preterm tissues is the presence of huge amounts of stem/progenitor cells in multiple organs, including kidney, brain, heart, adrenals, and lungs. In other organs, such as liver, the completely different burden of cell types in preterm infants is mainly related to the different function of the liver during gestation, mainly focused on hematopoiesis, a function that is taken by bone marrow after birth. Our preliminary studies showed that the antigens expressed by stem/progenitors differ significantly from one organ to the next. Moreover, within each developing human tissue, reactivity for different stem cell markers also changes during gestation, according with the multiple differentiation steps encountered by each progenitor during development. A better knowledge of stem/progenitor cells of preterms will allow neonatologists to boost preterm organ maturation, favoring the differentiation of the multiple cells types that characterize each organ in at term neonates.""",
]

#### Example: 1

In [17]:
input_data = {
    "input_text": covid19_articles,
    "params": {"template": "summarization", "temperature":0.2}
}

# The timeout for InvokeEndpoint is 60 seconds, so please carefully select the number of input texts you are sending in one request.
data = invoke_realtime_endpoint(input_data)
pd.DataFrame(data)

,response
0,"The research paper examines the impact of COVID-19 on elderly individuals in Brazil during the period from August 16th to December 26th, 2020. The study involved analyzing data from 1544 confirmed cases of COVID-19 among people aged 60 years and above.\n\nKey findings include:\n\n* A high mortality rate of 48% among patients who required intensive care unit admission due to severe illness caused by SARS-CoV-2.\n* Several factors that contribute to an increased risk of death were identified including immunodeficiency (188% higher odds), kidney disease (166%), neurological diseases (103%), difficulty breathing (dyspnea - 86%), lung conditions (pneumopathy - 55%), low oxygen levels (<95% - 53%), respiratory distress (49%), advanced age (increased by 5% per year), having a sore throat (31%) and being male (50% lower odds compared to females).\n* Common pre-existing health issues such as heart disease and diabetes did not show any significant association with an elevated risk of dying from COVID-19.\n\nThese results highlight important considerations for healthcare providers when managing COVID-19 infections in older populations within this demographic group. It emphasizes the need for close monitoring and targeted interventions based on specific risk factors."
1,"The objective of this study was to compare the outcomes of conservative management in cases of early-onset severe preeclampsia between randomized controlled trials and observational studies. A total of 44 studies were analyzed, including five randomized ones. It was found that while there was no significant difference in the duration of pregnancy prolongation between the two types of studies, maternal and perinatal complications were more frequent in observational studies compared to randomized trials. Additionally, the rate of cesarean sections was higher in randomized studies. Overall, it can be concluded that observational studies involving conservative management of early-onset preeclampsia without fetal growth restriction or HELLP syndrome have higher rates of adverse outcomes for mothers and newborns."
2,"The research paper discusses efforts towards eliminating Gambiense Human African Trypanosomiasis (gHAT), a parasitic disease transmitted by tsetse flies, in the Mandoul region of Chad. The authors used mathematical modeling based on historical case data from 2000 to 2019 to evaluate progress made in controlling the spread of the disease. They found significant improvement in diagnosis at healthcare facilities leading to higher detection rates. Additionally, they determined that the diagnostic algorithm for active screening had high specificity but suggested performing further confirmation tests due to potential false positives among recently reported cases. The study concluded that intensive intervention measures resulted in interrupting transmission around 2015. It recommended continued surveillance and more accurate case measurements before stopping active screening and vector control activities. Overall, the findings indicate substantial progress toward gHAT elimination in the Mandoul area."
3,"The microanatomy of most human organs differs between newborns and adults, particularly in preterm babies who have less matured organs and ongoing cell differentiation. Preterm tissues contain large numbers of stem/progenitor cells across various organs like kidneys, brains, hearts, adrenal glands, and lungs. The expression of antigens by these stem/progenitor cells varies among organs and changes throughout gestation based on developmental stages. Understanding these cells can help improve preterm organ maturation and promote proper cell type differentiation."


Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [20]:
import json
import os

# JSON file names
validation_json_file_name1 = "input1.json"
validation_json_file_name2 = "input2.json"

# JSONL file names
validation_jsonl_file_name1 = "input1.jsonl"
validation_jsonl_file_name2 = "input2.jsonl"


# JSON paths
validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/json/"

# JSONL paths
validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/jsonl/"


def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )


In [21]:
def create_jsonl(records, params=None):
    if isinstance(records, str):
        records = [records]

    json_records = [
        {"input_text": text} if params is None else {"input_text": text, "params": params}
        for text in records
    ]

    return '\n'.join(json.dumps(record, ensure_ascii=False) for record in json_records)


# Example data for JSONL files
input_jsonl_data1 = create_jsonl(open_book_qa, params = {"template": "open_book_qa"})
input_jsonl_data2 = create_jsonl(covid19_articles, params = {"template": "summarization"})

# Example data for JSON files
input_json_data1 = json.dumps({"input_text": open_book_qa, "params": {"template": "open_book_qa"}})
input_json_data2 = json.dumps({"input_text": covid19_articles, "params": {"template": "summarization"}})


# Write and upload JSONL data
write_and_upload_to_s3(input_jsonl_data1, f"{validation_jsonl_file_name1}")
write_and_upload_to_s3(input_jsonl_data2, f"{validation_jsonl_file_name2}")

# Write and upload JSON data
write_and_upload_to_s3(input_json_data1, f"{validation_json_file_name1}")
write_and_upload_to_s3(input_json_data2,  f"{validation_json_file_name2}")

### JSON

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path,
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [23]:
from urllib.parse import urlparse

def retrieve_json_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data)
    display(df)


#### Example 1: Question Answering

In [24]:
# open_book_qa
retrieve_json_output_from_s3(validation_json_file_name1)

[01/28/25 06:42:39] INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=621410;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=666379;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py#481\481]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

,response
0,The patient is concerned about potential long-term side effects of radiation therapy.
1,"After experiencing a sharp pain in his lower back while lifting boxes during work, the patient also reported having radiating symptoms down his right buttock extending to the lateral side of his leg below the knee, which worsened over time. These symptoms suggest possible lumbar radiculopathy or sciatica due to nerve compression associated with disc herniation or other spinal pathology. It would be important to further evaluate these complaints through physical examination and potentially imaging studies such as MRI if necessary."
2,The primary issue reported by the patient is recent worsening of her hypertension.


#### Example 2: Summarization

In [25]:
retrieve_json_output_from_s3(validation_json_file_name2)

[01/28/25 06:42:40] INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=888999;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=618956;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py#481\481]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

,response
0,"The research paper aimed to investigate the impact of COVID-19 on elderly individuals in Brazil during specific time periods in 2020. It analyzed data from over 1500 confirmed cases of COVID-19 among people aged 60 years old or above. The findings indicated that almost half of those who required intensive care unit admission due to severe illness caused by the virus succumbed to it. Various factors such as advanced age, certain pre-existing health conditions like kidney disease, neurological disorders, low oxygen levels, breathing difficulties, and even having a sore throat contributed to higher mortality rates. Interestingly, common co-morbidities such as heart diseases and diabetes were prevalent but did not directly lead to increased chances of dying from COVID-19. Overall, understanding these risk factors can help healthcare professionals provide better support and interventions tailored specifically towards vulnerable populations affected by this pandemic."
1,"This study aimed to compare the outcomes of expectant management in cases of early-onset severe preeclampsia between randomized controlled trials (RCTs) and observational studies. A total of 44 studies published from 1990 to 2018 were analyzed, including five RCTs. Results showed that while there was no significant difference in pregnancy prolongation duration between the two types of studies, observational studies had a higher incidence of maternal and perinatal complications compared to RCTs. Additionally, RCTs reported a higher rate of cesarean deliveries. Overall, it can be concluded that observational studies on expectant management of early-onset preeclampsia without fetal growth restriction or HELLP syndrome have poorer maternal and perinatal outcomes."
2,"The research paper discusses the progress made towards eliminating gambiense human African trypanosomiasis (gHAT), commonly known as sleeping sickness, in the Mandoul disease focus of Chad. The implementation of an integrated program combining vector control, screening, and treatment has significantly reduced the number of reported cases over the years. Using historical case data from 2000 to 2019, along with updates to their mathematical model including improved diagnostic specificity and new fitting methods, the researchers determined that while transmission interruption may have occurred around 2015 due to intensive interventions, ongoing efforts should include more accurate confirmation testing. This will ensure careful monitoring during the final stages of eradication efforts and provide clearer guidance on when active screening and vector control measures can safely cease. Additionally, they found evidence suggesting that recent reported cases could potentially be false positives given the extremely low prevalence of actual infections. Overall, continued vigilance and precise surveillance strategies remain crucial to achieving complete elimination goals."
3,"The microanatomy of most human organs differs between newborns and adults, particularly in preterm babies where there's less maturity and ongoing cell differentiation. Preterm tissues have high numbers of stem/progenitor cells across various organs like kidneys, brains, hearts, adrenal glands, and lungs. Organs like livers show different cell types primarily involved in producing blood before switching this role post-birth to the bone marrow. Stem/progenitor cells express varying antigens based on the specific organ they're found in, which change throughout fetal development. Understanding these cells can help improve preterm organ growth and promote proper differentiation into mature cell types seen in full-term babies. This information could aid neonatologists in boosting preterm organ maturation through targeted interventions."


### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path,
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [27]:
from urllib.parse import urlparse

def retrieve_jsonlines_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)


#### Example 1: Question Answering

In [28]:
# open_book_qa
retrieve_jsonlines_output_from_s3(validation_jsonl_file_name1)

[01/28/25 07:39:08] INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=203923;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=845999;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py#481\481]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

{"response": "The patient is concerned about potential long-term side effects of radiation therapy."}
{"response": "After experiencing a sharp pain in his lower back while lifting boxes during work, the patient also reported having radiating symptoms down his right buttock extending to the lateral side of his leg below the knee, which worsened over time. These symptoms suggest possible lumbar radiculopathy or sciatica due to nerve compression associated with disc herniation or other spinal pathology. It would be important to further evaluate these complaints through physical examination and potentially imaging studies such as MRI if necessary."}
{"response": "The primary issue reported by the patient is recent worsening of her hypertension."}


#### Example 2: Summarization

In [29]:
retrieve_jsonlines_output_from_s3(validation_jsonl_file_name2)

                    INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=632030;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=411072;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/httpchecksum.py#481\481]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

{"response": "The research paper aimed to investigate the impact of COVID-19 on elderly individuals in Brazil during specific time periods in 2020. It analyzed data from over 1500 confirmed cases of COVID-19 among people aged 60 years old or above. The findings indicated that almost half of those who required intensive care unit admission due to severe illness caused by the virus succumbed to it. Various factors such as advanced age, certain pre-existing health conditions like kidney disease, neurological disorders, low oxygen levels, breathing difficulties, and even having a sore throat contributed to higher mortality rates. Interestingly, common co-morbidities such as heart diseases and diabetes were prevalent but did not directly lead to increased chances of dying from COVID-19. Overall, understanding these risk factors can help healthcare professionals provide better support and interventions tailored specifically towards vulnerable populations affected by this pandemic."}
{"respon

In [ ]:
model.delete_model()

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

